In [41]:
from unstructured.partition.utils.sorting import sort_page_elements
!pip install unstructured pi_heif unstructured_inference unstructured_pytesseract
!pip install langdetect
!pip install --upgrade pymupdf

In [53]:
# importing required modules  
from langdetect import detect
import pymupdf
import PyPDF2
import ocrmypdf
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition

In [45]:
texto = "Olá, tudo bem? Como vai a família?"
lang = detect(texto)
lang

'pt'

# PyPDF2

In [64]:
# pdf = '018368000101011.pdf'
# pdf = '676706.pdf'
# pdf = 'CRA023005V5-ASS28082024V01-000727267.pdf'
# pdf = 'ALLD_6_Escritura - Escritura_20231206_000.pdf'
# pdf = '118498.pdf'
# pdf = "CRA023005V5-ASS28082024V01-000727267_ocr.pdf"
pdf = '573735.pdf'

pdf_ocr = f"{pdf.split('.')[0]}_ocr.{pdf.split('.')[-1]}"
print(pdf_ocr)

pdfFileObj = open(pdf, 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print(len(pdfReader.pages)) # will give total number of pages in pdf
print(pdfReader.is_encrypted)
# pdfReader.xmp_metadata.custom_properties

573735_ocr.pdf
127
False


# PyMuPDF

In [7]:
import subprocess,os
# subprocess.call(["export", "TESSDATA_PREFIX=/usr/share/tesseract-ocr/4.00/tessdata"])
os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata"

In [47]:
doc = pymupdf.open(pdf) # open a document
print(len(doc))
for page in doc: # iterate the document pages
    imgs = page.get_images()
    print(len(imgs))
    
    # text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
    # print(text)
    tp = page.get_textpage_ocr()
    text2 = page.get_text(textpage=tp)
    print(text2)


2
2
              Caderno Publicações Digitais  
              Diário do Acionista, 28 de agosto de 2024 
 
 
 
 
 
 
 
 
 
JORNAL DIARIO DO 
ACIONISTA 
LTDA:26530904000112
Assinado de forma digital por 
JORNAL DIARIO DO ACIONISTA 
LTDA:26530904000112 
Dados: 2024.08.27 19:41:54 -03'00'
CANAL COMPANHIA DE SECURITIZACAO
CNPJ/ME n° 41 811.375/0001-19 - NIRE 353.0057653-5
¢
EDITAL DE CONVOCACAO DA ASSEMBLEIA ESPECIAL DE TITULARES DOS CERTIFICADOS DE
RECEBIVEIS DO AGRONEGOCIO DA SERIE UNICA DA 42° EMISSAO DE CERTIFICADOS DE
RECEBIVEIS DO AGRONEGOCIO DA CANAL COMPANHIA DE SECURITIZACAO
Ficam convocados 0s titulares dos Certificados de Certificados de Recebiveis do Agronegécio da Série Unica da 42°
Emissdo da CANAL COMPANHIA DE SECURITIZAGAO, sociedade por aces com sede na Cidade de Sao Paulo, Estado
de Sao Paulo, na Rua Prof. Atilio Innocenti, 474, conjuntos 1009 e 1010, CEP 04.538-001, inscrita no Cadastro Nacional
da Pessoa Juridica ("CNPJ") sob o n° 41.811.375/0001-19, neste ato represen

In [48]:
lang = detect(text2)
lang

'pt'

In [15]:
pageObj = pdfReader.pages[0]
pageObj['/Resources']['/Font'].keys()

KeyError: '/Font'

In [46]:
text=(pageObj.extract_text())
text=text.split(",")
text

['']

In [49]:
info = ocrmypdf.pdfinfo.PdfInfo(pdf)

In [50]:
info.has_acroform

True

In [38]:
info.has_signature

False

In [39]:
info.needs_rendering

False

In [5]:
ocrmypdf.ocr(pdf, pdf_ocr, force_ocr=True, optimize=3,
                         invalidate_digital_signatures=True,
                         language="por", output_type="pdf", 
                         quiet=True, progress_bar=False)

<ExitCode.ok: 0>

# Unstructured-IO

In [71]:
# infer_table_structure=True automatically selects hi_res strategy
elements = partition(filename=pdf, strategy='hi_res', include_page_breaks=True, extract_image_block_types=['Image'])
elements

 ...]

In [73]:
print("\n\n".join([str(el) for el in elements]))

DA Autorregulacao ANBIMA PN dae ler lace}

TERMO DE SECURITIZAÇÃO DE DIREITOS CREDITÓRIOS DO AGRONEGÓCIO

para emissão de

CERTIFICADOS DE RECEBÍVEIS DO AGRONEGÓCIO DA 1ª E 2ª SÉRIES DA 34ª (TRIGÉSIMA QUARTA) EMISSÃO DA

CERES SECURITIZADORA

CERES SECURITIZADORA S.A. como companhia securitizadora

lastreados em direitos creditórios do agronegócio devidos pela NATIVA AGRONEGÓCIOS LTDA.

celebrado com

OLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A. como agente fiduciário

Datado de 18 de dezembro de 2023

Documento assinado no Assinador Registro de Iméveis. Para validar 0 documento e suas assinaturas acesse https://assinador.registrodeimoveis.org.br/validate/546KQ-TBSM7-VTY3M-6YDA8.

—_ @ vegistro imévels do Brasi



TERMO DE SECURITIZAÇÃO DE DIREITOS CREDITÓRIOS DO AGRONEGÓCIO PARA EMISSÃO DE CERTIFICADOS DE RECEBÍVEIS DO AGRONEGÓCIO, DA 1ª E 2ª SÉRIES, DA 34ª (TRIGÉSIMA QUARTA) EMISSÃO DA CERES SECURITIZADORA S.A., LASTREADOS EM DIREITOS CREDITÓRIOS DO AGRONEGÓCIO 

In [76]:
elements[0].text

'DA Autorregulacao ANBIMA PN dae ler lace}'

In [74]:
import json, base64, io
from PIL import Image

image_data = base64.b64decode(elements[0])
image = Image.open(io.BytesIO(image_data))
image.show()

TypeError: argument should be a bytes-like object or ASCII string, not 'Image'